In [4]:
# Importing required libraries
import pandas as pd
import time

from webdriver_manager.microsoft import EdgeChromiumDriverManager

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [7]:
# Initializing the driver
#driver = webdriver.Edge(EdgeChromiumDriverManager().install())
driver = webdriver.Firefox(executable_path=r"C:\Users\Amisha\OneDrive\Aegis\NLP\Project\geckodriver.exe")
#driver = webdriver.Chrome(r"C:\Users\Amisha\OneDrive\Aegis\NLP\Project\chromedriver.exe")
# Browsing to billboard hot-100 chart on the driver
#driver.get("https://www.billboard.com/charts/hot-100/")
dates = [str(i) for i in range(2012,2023)]
#driver.get("https://www.google.com")

In [8]:
import time

artists_data = {} # List to store name of artists
songs_data = {} # List to store top 100 songs

for j in dates:
    link = "https://www.billboard.com/charts/hot-100/{}-07-01/".format(j)
    driver.implicitly_wait(10)
    driver.get(link)

    # Container for referencing the division with top 100 songs
    div = driver.find_elements_by_class_name('o-chart-results-list-row-container')

    songs, artists = [], []
     
    # Fetching the top 100 songs
    for i in div:
        # Getting the song name
        song = i.find_element_by_css_selector('li ul li h3').text
        # Getting the artist name
        name = i.find_element_by_css_selector('li ul li span').text
        # Appending the song and artist name to the lists
        songs.append(song)
        artists.append(name)
        
    artists_data[j] = artists
    songs_data[j] = songs

In [12]:
#artists_data

In [13]:
#songs_data

In [11]:
# Terminating a session and closing the active browser
driver.close()

In [28]:
# Initializing the driver
#driver = webdriver.Edge(EdgeChromiumDriverManager().install())
driver = webdriver.Firefox(executable_path=r"C:\Users\Amisha\OneDrive\Aegis\NLP\Project\geckodriver.exe")

In [29]:
# Browsing to Google
driver.get("https://www.google.com")

In [30]:
# Dataframe to store the song, artist and lyrics
df = pd.DataFrame(columns = ['Top100Year','SongTitle','Artist','LyricsStatus','Lyrics','ReleaseYear','Genre'])

# Fetching song lyrics for top 100 songs
for year in songs_data.keys():
    songs_list = songs_data[year]
    artists_list = artists_data[year]
    # Index Variable
    a = 0
    for song in songs_list:
        # Selenium WebDriverWait is one of the Explicit waits.
        # Explicit waits are confined to a particular web element. 
        # Explicit Wait is code you define to wait for a certain condition to occur before proceeding further in the code.
        # It is applied on certain element with defined expected condition and time. 
        # This wait is only applied to the specified element. This wait can also throw exception when element is not found.
        # Here we will wait for the google search button element to be clickable.
        # https://www.softwaretestingmaterial.com/webdriverwait-selenium-webdriver/
        element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, "q")))
        # Entering the search text
        element.send_keys("{} {} lyrics".format(artists_list[a],song))
        # Submiting the form for google search
        element.submit()
        # To prevent captcha being initialized due to bot activity
        time.sleep(10)
        try:
            # Fetching the default lyrics fetched by google using 'lyricfind'
            lyric = driver.find_element_by_class_name('xaAUmb').text
            # If default lyrics is available, set lyricstatus to True
            # Lyric Status True indicates lyrics are available
            status = True
        except:
            # If default lyrics is not available, set lyricstatus to False
            # Lyric Status False indicates lyrics are not available
            status = False
            lyric = None
        try:
            # Fetching genre and release year of the song
            info = driver.find_elements_by_css_selector('div.rVusze')
            for i in info:
                if i.find_element_by_class_name('w8qArf').text == 'Released:':
                    release_year = i.find_element_by_class_name('LrzXr').text
                if i.find_element_by_class_name('w8qArf').text == 'Genres:':
                    genre = i.find_element_by_class_name('LrzXr').text
            if len(release_year) == 0:
                release_year = None
            if len(genre) == 0:
                genre = None
        except:
            # If genre and year information is not available
            release_year = None
            genre = None
        # Adding the song, artist and lyrics to dataframe
        df.loc[len(df.index)] = [year, song, artists_list[a], status, lyric, release_year, genre] 
        a+=1
        # Browsing back to Google search home page
        driver.back()

In [31]:
# Terminating a session and closing the active browser
driver.close()

In [32]:
df

,Top100Year,SongTitle,Artist,LyricsStatus,Lyrics,ReleaseYear,Genre
0,2012,Call Me Maybe,Carly Rae Jepsen,True,"I threw a wish in the well\nDon't ask me, I'll...",2012,"R&B/Soul, Pop"
1,2012,Payphone,Maroon 5 Featuring Wiz Khalifa,True,I'm at a payphone trying to call home\nAll of ...,2012,"R&B/Soul, Pop"
2,2012,Somebody That I Used To Know,Gotye Featuring Kimbra,True,Now and then I think of when we were together\...,2011,"R&B/Soul, Pop"
3,2012,Wide Awake,Katy Perry,True,I'm wide awake\nI'm wide awake\nI'm wide awake...,2012,"R&B/Soul, Pop, UK R&B"
4,2012,Where Have You Been,Rihanna,True,"I've been everywhere, man\nLooking for someone...",2011,"Dance-pop, R&B/Soul, Electro house, Pop, UK R&B"
...,...,...,...,...,...,...,...
1095,2022,N95,Kendrick Lamar,True,"Hello, new world, all the boys and girls\nI go...",2022,"Pop, Country"
1096,2022,Love Me More,Sam Smith,True,Have you ever felt like being somebody else?\n...,2022,"R&B/Soul, Pop"
1097,2022,New Truck,Dylan Scott,True,I need a new truck\nThere ain't nothing wrong ...,2022,"R&B/Soul, Pop"
1098,2022,She's All I Wanna Be,Tate McRae,True,You want the girl with the small waist\nAnd th...,2022,"R&B/Soul, Pop"


In [33]:
# Converting DataFrame to CSV
df.to_csv('billboard_2012_to_2022_top_100_song_lyrics.csv', index=False)